In [1]:
import pandas as pd
import numpy as np
import re
import requests 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import selenium.webdriver.support.ui as ui
import selenium.webdriver.support.expected_conditions as EC
import os
import time
import datetime

In [71]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')
dir_path = os.path.dirname(os.path.realpath('chromedriver'))
chromedriver = dir_path + '/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver

In [72]:
links = pd.read_csv('accident_links.csv')

In [73]:
driver = webdriver.Chrome(executable_path=chromedriver)

In [74]:
url = links.link[9]
url


'https://app.myaccident.org/accident/00000117157155'

In [75]:
list_of_dicts = []

In [76]:
driver.get(url)
ui.WebDriverWait(driver, 15).until(EC.visibility_of_all_elements_located((By.ID, 'ACCIDENT')))

[<selenium.webdriver.remote.webelement.WebElement (session="b85261d49c21c44d0108b6f9bf9355fb", element="7fb5373d-bf47-4733-8737-5e4c47274236")>]

In [77]:
containers = driver.find_elements_by_id('accident-top')

In [78]:
for container in containers:
    variables = container.text.split('\n')
    caseId = variables[1].split(':')
    crashId = variables[2].split(':')

In [79]:
containers = driver.find_elements_by_id('accident-header')

In [80]:
for container in containers:
    variables = container.text.split('\n')
    city = variables[1]
    date = variables[3]
    Police = variables[5]

In [81]:
containers = driver.find_elements_by_id('conditions')

In [82]:
for container in containers:
    variables = container.text.split('\n')
    if variables[variables.index('Road & Traffic Conditions') + 1] == 'Weather':
            variables.insert(variables.index('Road & Traffic Conditions') + 1, np.nan)
    if variables[-1] == 'Weather':
            variables.insert(variables.index('Weather')+1, np.nan)
    trafficConditions = variables[variables.index('Road & Traffic Conditions') + 1]
    weatherConditions = variables[variables.index('Weather') + 1]
    
    

In [83]:
containers = driver.find_elements_by_class_name('row')

In [84]:
summary = containers[0].text.split('\n')
description = containers[1].text.split('\n')  

In [85]:
accident_factor = ''
for i in range(len(summary)):
    if i > 0 and i <= summary.index('Date & Time Of Crash'):
        accident_factor += summary[i] + ' '
speedlimit = summary[summary.index("Speed Limit") + 1]
accidentLocation = summary[-1]
injuryNumber = summary[summary.index('Total Number of Injuries')-1]
numberofVehicles = summary[summary.index('Total Number of Vehicles') -1]
numberofOccupants = summary[summary.index('Total Number of Occupants')-1]

In [86]:
containers = driver.find_elements_by_class_name('driver-vehicle')

In [87]:
for container in containers:
         #(ignore compensation text box)'
        if len(container.find_elements_by_class_name('at-fault')) == 0:
            atFault = 0
        else:
            atFault = 1
        variables = container.text.split('\n')
        if 'Based on' in variables[0] and 'Accident' in variables[1]:
            variables.pop(0)
            variables.pop(0)  
        car = variables[0].split(',')
        carColor = car[1]
        if car[0] == '':
            carMake = np.nan
            carYear = np.nan
        else:
            makeYear = car[0].split('-')
            carMake = makeYear[0]
            carYear = makeYear[1]
        #if the next index is the next key and not a value for the current key, then the value is missing, fill with nan
        if variables[variables.index('Age') + 1] == 'Gender':
            variables.insert(variables.index('Age') + 1, np.nan)
        if variables[variables.index('Gender') + 1] == "Ethnicity":
            variables.insert(variables.index('Gender')+1, np.nan)
        if variables[variables.index('Ethnicity') + 1] == "Residence Of":
            variables.insert(variables.index('Ethnicity')+1, np.nan)
        if variables[variables.index('Residence Of') + 1] == "Damage Area":
            variables.insert(variables.index('Residence Of')+1, np.nan)
        if variables[variables.index('Damage Area') + 1] == "Driver License Type":
            variables.insert(variables.index('Damage Area')+1, np.nan)
        if variables[variables.index('Driver License Type') + 1] == "Vehicle License State ID":
            variables.insert(variables.index('Driver License Type')+1, np.nan)
            #if police did not get vin number, the incident was more than likely a hit and run, as no info from the car other car was gathered as well
        if 'VIN' not in variables:
            driverHitAndrun = 1
            variables.insert(variables.index('Vehicle License State ID')+1, 'VIN') 
        else:
            driverHitAndrun = 0
        if variables[variables.index('Vehicle License State ID') + 1] == "VIN":
            variables.insert(variables.index('Vehicle License State ID')+1, np.nan)
        if variables[variables.index('VIN') + 1] == "Insured":
            variables.insert(variables.index('VIN')+1, np.nan)
        if variables[variables.index('Insured') + 1] == "Towing Company":
            variables.insert(variables.index('VIN')+1, np.nan)
        if variables[-1] == 'Towing Company':
            variables.insert(variables.index('Towing Company')+1, np.nan)
            
        #now that the index lengths are uniform, we can assign the values
        driverAge = variables[3]
        driverGender = variables[5]
        driverEthnicity = variables[7]
        driverResidence = variables[9]
        driverCarDamage = variables[11]
        driverLicenseType = variables[13]
        driverLicenseState = variables[15]
        drivercarVIN = variables[17]
        driverInsured = variables[19]
        
        #store all values into a dictionary here, all this information is one observation
        dictionary = {'case_id': caseId[1], 'crash_id' : crashId[1], 'crash_city': city, 'crash_date':date, 'police_dept': Police, 'accident_factor': accident_factor, 'speed_limit' : speedlimit \
                      ,'crash_location': accidentLocation, 'num_of_injuries': injuryNumber, 'num_of_vehicles': numberofVehicles , 'num_of_occupants': numberofOccupants, 'at_fault' : atFault \
                      ,'hit_and_run': driverHitAndrun, 'car_make':carMake, 'car_year': carYear, 'car_color': carColor, 'driver_age':driverAge, 'driver_gender': driverGender, 'driver_ethnicity' : driverEthnicity \
                      ,'driver_residence': driverResidence,'driver_car_damage':driverCarDamage,'driver_license_type':driverLicenseType, 'driver_license_state': driverLicenseState, 'driver_insured': driverInsured }
        
        list_of_dicts.append(dictionary)

In [88]:
df = pd.DataFrame(list_of_dicts)

In [89]:
df

,case_id,crash_id,crash_city,crash_date,police_dept,accident_factor,speed_limit,crash_location,num_of_injuries,num_of_vehicles,...,car_year,car_color,driver_age,driver_gender,driver_ethnicity,driver_residence,driver_car_damage,driver_license_type,driver_license_state,driver_insured
0,SAPD19138777,17157155,"SAN ANTONIO, TX",06/30/19 05:10 PM,SAN ANTONIO POLICE DEPARTMENT,The driver of 2005 GMC FOLLOWED TOO CLOSELY Da...,35 MPH,"JACKSON KELLER RD 2216, San Antonio, Bexar, TX...",0,2,...,2005,Gray,18,Male,Hispanic,NaN,FC - FRONT END DAMAGE CONCENTRATED IMPACT,"Id card, unlicensed",Texas,Yes
1,SAPD19138777,17157155,"SAN ANTONIO, TX",06/30/19 05:10 PM,SAN ANTONIO POLICE DEPARTMENT,The driver of 2005 GMC FOLLOWED TOO CLOSELY Da...,35 MPH,"JACKSON KELLER RD 2216, San Antonio, Bexar, TX...",0,2,...,2017,White,68,Female,Hispanic,NaN,BD - BACK END DAMAGE DISTRIBUTED IMPACT,"Driver license, class c",Texas,Yes


In [90]:
links = pd.read_csv('accident_links.csv')

In [91]:
links.shape

(47380, 1)

In [94]:
links.drop_duplicates().shape

(46785, 1)

In [95]:
driver.close()

In [93]:
#will drop duplicates as we itterate through each page and store the contents in anquire2.py

In [8]:
links = pd.read_csv('accident_links_continued.csv')

In [9]:
links = links.iloc[1: , :]
links.to_csv('accident_links_continued.csv',index=False)